In [128]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import pandas as pd
import numpy as np

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

In [127]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
churnData= pd.read_csv('files_for_lab/Customer-Churn.csv')

In [37]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [45]:
churnData['TotalCharges'].iloc[488]

' '

In [47]:
def replacing_white(x):
    if x == ' ':
        x = '0'
        return x
    else :return x

In [48]:
churnData['TotalCharges']= churnData['TotalCharges'].apply(replacing_white)

In [49]:
churnData['TotalCharges']= pd.to_numeric(churnData.TotalCharges)

In [56]:
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.50,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [60]:
churn= churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges', 'Churn']]

In [61]:
churn.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,1,0,29.85,29.85,No
1,34,0,56.95,1889.50,No
2,2,0,53.85,108.15,Yes
3,45,0,42.30,1840.75,No
4,2,0,70.70,151.65,Yes


In [96]:
features= churn.drop(columns= 'Churn')
target= churn[['Churn']]

In [97]:
scaler = MinMaxScaler()
features_scaled= scaler.fit_transform(features)
features_scaled= pd.DataFrame(features_scaled, columns= ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges'])

In [98]:
features_scaled

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,0.013889,0.0,0.115423,0.003437
1,0.472222,0.0,0.385075,0.217564
2,0.027778,0.0,0.354229,0.012453
3,0.625000,0.0,0.239303,0.211951
4,0.027778,0.0,0.521891,0.017462
...,...,...,...,...
7038,0.333333,0.0,0.662189,0.229194
7039,1.000000,0.0,0.845274,0.847792
7040,0.152778,0.0,0.112935,0.039892
7041,0.055556,1.0,0.558706,0.035303


In [168]:
X_train, X_test, y_train, y_test = train_test_split(over_churn, target_2)

In [169]:
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='ovr').fit(X_train, y_train)

In [170]:
predictions = classification.predict(X_test)
print(classification.score(X_test, y_test))
print(predictions)
# around 78.3% NORMAL

# undersampled: 71.8%

# over: 72.22%

# mix: Too late to try this now.

0.7224584460765365
['Yes' 'Yes' 'No' ... 'Yes' 'Yes' 'Yes']


In [103]:
target['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [107]:
len(target)

7043

In [149]:
#down
cat_N= target[target['Churn'] == 'No']
cat_Y= target[target['Churn'] == 'Yes']

In [134]:
#cat_N.head()

In [150]:
len(cat_N)
len(cat_Y)

5174

1869

In [151]:
cat_N_undersampled = resample(cat_N, 
                                   replace=False, 
                                   n_samples = len(cat_Y))

In [152]:
print(cat_N_undersampled.shape)
print(cat_Y.shape)

(1869, 1)
(1869, 1)


In [153]:
type(cat_N_undersampled)
type(cat_Y)

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

In [123]:
target_under= pd.concat([cat_N_undersampled, cat_Y], axis=0)

In [154]:
under_churn= features_scaled.join(target_under, how= 'right')

In [139]:
target_1= under_churn['Churn']
under_churn= under_churn.drop(columns= 'Churn')

In [144]:
#over
cat_N_over= target[target['Churn'] == 'No']
cat_Y_over= target[target['Churn'] == 'Yes']

In [145]:
cat_Y_oversampled = resample(cat_Y_over, 
                                   replace=True, 
                                   n_samples = len(cat_N_over))

In [147]:
print(cat_Y_oversampled.shape)
print(cat_N_over.shape)

(5174, 1)
(5174, 1)


In [148]:
target_over= pd.concat([cat_Y_oversampled, cat_N_over], axis=0)

In [165]:
over_churn= features_scaled.join(target_over, how= 'right')

In [167]:
target_2= over_churn['Churn']
over_churn= over_churn.drop(columns= 'Churn')

In [166]:
len(over_churn)

10348